In [100]:
import pandas as pd
import os
import urllib
import json
from tqdm import tqdm

# Get Data

In [115]:
ROOT = os.path.abspath(os.getcwd())
df = pd.read_excel(f"{ROOT}/data/meta.xlsx", sheet_name=None)["analysed_data"]
print(df)

              강산업 물품      단가 Unnamed: 2
0            강력 20kg   27800        NaN
1         영양중력분 20kg   25000        NaN
2            박력 20kg   26500        NaN
3            설탕 15kg   20200        NaN
4            꽃소금 3kg    4800        NaN
..               ...     ...        ...
400          피자스페치 대    5500        NaN
401          잠봉 500g   14100        NaN
402              NaN     NaN        NaN
403  아몬드분말버크 11.34kg  117360        NaN
404        냉동난백액 9kg   61000        NaN

[405 rows x 3 columns]


In [116]:
df.drop(["Unnamed: 2"], axis=1, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)

              강산업 물품      단가
0            강력 20kg   27800
1         영양중력분 20kg   25000
2            박력 20kg   26500
3            설탕 15kg   20200
4            꽃소금 3kg    4800
..               ...     ...
365          피자스페치 중    5200
366          피자스페치 대    5500
367          잠봉 500g   14100
368  아몬드분말버크 11.34kg  117360
369        냉동난백액 9kg   61000

[370 rows x 2 columns]


In [117]:
# 확인용 코드
print(len(df)-len(df.drop_duplicates(subset=["강산업 물품"])))



Dup = {}
Dup_idx = []
for value in df["강산업 물품"]:
    Dup[value.split(" ")[0]] = 0

for idx, value in enumerate(df["강산업 물품"]):
    if Dup[value.split(" ")[0]] == 0:
        Dup[value.split(" ")[0]] = 1
    else:
        Dup_idx.append(idx)
        Dup[value.split(" ")[0]] += 1

Dup_len = 0
for name, value in Dup.items():
    if value > 1:
        Dup_len += 1

print(Dup_len)

55
55


In [118]:
df.drop_duplicates(subset=["강산업 물품"], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)

              강산업 물품      단가
0            강력 20kg   27800
1         영양중력분 20kg   25000
2            박력 20kg   26500
3            설탕 15kg   20200
4            꽃소금 3kg    4800
..               ...     ...
310         사각케익2호받침     400
311         사각케익2호뚜껑     950
312          피자스페치 중    5200
313          피자스페치 대    5500
314  아몬드분말버크 11.34kg  117360

[315 rows x 2 columns]


In [123]:
re_idx = []
for idx, value in enumerate(df["단가"]):
    if isinstance(value, str):
        re_idx.append(idx)
        print(value)
print("--------------------------")
print("total to remove: ", len(re_idx))

보류 22쪽
보류 23쪽
보류 8
보류 3300
보류 9300
보류 7100
보류 7100
보류 7500
보류
보류 148900
보류 4500
--------------------------
total to remove:  11


In [124]:
df.drop(re_idx, inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)

              강산업 물품      단가
0            강력 20kg   27800
1         영양중력분 20kg   25000
2            박력 20kg   26500
3            설탕 15kg   20200
4            꽃소금 3kg    4800
..               ...     ...
299         사각케익2호받침     400
300         사각케익2호뚜껑     950
301          피자스페치 중    5200
302          피자스페치 대    5500
303  아몬드분말버크 11.34kg  117360

[304 rows x 2 columns]


In [125]:
ROOT = os.path.abspath(os.getcwd())
for name in tqdm(df["강산업 물품"]):
    query = urllib.parse.quote(name)

    url = f"https://openapi.naver.com/v1/search/shop.json?query={query}&display=100"
    client_id = "4OQMv3YxrKsVojrlewHN"
    client_secret = "v7mCeaUi1B"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    response = response.read().decode("utf-8")
    response = json.loads(response)

    with open(f"{ROOT}/data/{name}.json", "w") as f:
        json.dump(response, f)

100%|██████████| 304/304 [01:37<00:00,  3.13it/s]


# Analysis

In [137]:
file = open(f"{ROOT}/data/description.txt", "w")

more_expensive = []
for name, value in zip(df["강산업 물품"], df['단가']):
    low_price = 100000000
    high_price = -1
    total_price = 0

    with open(f"{ROOT}/data/{name}.json", "r") as f:
        data = json.load(f)
        if 0 == len(data["items"]):
            break

        for item in data['items']:
            now_price = int(item['lprice'])
            total_price += now_price
            if high_price < now_price:
                high_price = now_price
            if low_price > now_price:
                low_price = now_price
    
    divider = len(data['items'])
    mean = total_price/divider
    diff = round(value-mean)
    if diff > 0:
        more_expensive.append(name)        
    file.write(f"\t{name}\n")
    file.write("----------------------------------------------------\n")
    file.write("diff(origin-mean):   {:+,}\n".format(diff))
    file.write("origin:              {:,}\n".format(value))
    file.write("mean:                {mean:,}   (samples: {samples})\n"
               .format(mean=round(mean), 
                       samples=divider))
    file.write("low:                 {:,}\n".format(low_price))
    file.write("high:                {:,}\n".format(high_price))
    file.write("####################################################\n")

file.close()

In [140]:
print("total more expensive items: ", len(more_expensive))
print("-------------------------------------------------")
for item_name in more_expensive:
    print(item_name)

total more expensive items:  23
-------------------------------------------------
에멘탈치즈가는쉬레드 750g
에멘탈크림치즈 1kg
피자치즈 2.5kg
코코아분말 1kg
건포도벌크 10kg
건조크랜베리클래식 11.34kg
몽블랑생지
타르트외피
파에테포요틴 2.5kg
파인소프트T 10kg
파인소프트C 2kg
아몬드프라린 5kg
오렌지스틱 1kg
스위스밸리크림치즈 13.6kg
조은밤홀 2kg
다이스밤 7L
피스타치오 1kg
냉동난백액 9kg
냉동참쑥 2kg
드롭밀크초콜렛 5kg
드롭화이트초콜렛 5kg
홍국쌀가루 대두
베이컨 1kg
